In [ ]:
stock_companies = {
    'ADANIPORTS': ['ADANIPORTS', 'Adani Ports'],
    'ASIANPAINT': ['ASIANPAINT', 'Asian Paints'],
    'AXISBANK': ['AXISBANK', 'Axis Bank'],
    'BAJAJ-AUTO': ['BAJAJ-AUTO', 'Bajaj Auto'],
    'BAJAJFINSV': ['BAJAJFINSV', 'Bajaj Finserv'],
    'BAJFINANCE': ['BAJFINANCE', 'Bajaj Finance'],
    'BHARTIARTL': ['BHARTIARTL', 'Bharti Airtel', 'Airtel'],
    'BRITANNIA': ['BRITANNIA', 'Britannia Industries'],
    'BPCL': ['BPCL', 'Bharat Petroleum'],
    'COALINDIA': ['COALINDIA', 'Coal India'],
    'EICHERMOT': ['EICHERMOT', 'Eicher Motors'],
    'DRREDDY': ['DRREDDY', "Dr. Reddy's Laboratories"],
    'GAIL': ['GAIL', 'Gas Authority of India'],
    'GRASIM': ['GRASIM', 'Grasim Industries'],
    'CIPLA': ['CIPLA', 'Cipla Limited'],
    'HCLTECH': ['HCLTECH', 'HCL Technologies'],
    'HDFC': ['HDFC', 'Housing Development Finance Corporation'],
    'HDFCBANK': ['HDFCBANK', 'HDFC Bank'],
    'HEROMOTOCO': ['HEROMOTOCO', 'Hero MotoCorp'],
    'HINDUNILVR': ['HINDUNILVR', 'Hindustan Unilever', 'HUL'],
    'HINDALCO': ['HINDALCO', 'Hindalco Industries'],
    'ICICIBANK': ['ICICIBANK', 'ICICI Bank'],
    'INDUSINDBK': ['INDUSINDBK', 'IndusInd Bank'],
    'INFY': ['INFY', 'Infosys'],
    'IOC': ['IOC', 'Indian Oil'],
    'ITC': ['ITC', 'ITC Limited'],
    'JSWSTEEL': ['JSWSTEEL', 'JSW Steel'],
    'KOTAKBANK': ['KOTAKBANK', 'Kotak Mahindra Bank'],
    'LT': ['LT', 'Larsen & Toubro'],
    'MARUTI': ['MARUTI', 'Maruti Suzuki'],
    'MM': ['MM', 'Mahindra & Mahindra'],
    'NESTLEIND': ['NESTLEIND', 'Nestle India'],
    'NTPC': ['NTPC', 'NTPC Limited'],
    'ONGC': ['ONGC', 'Oil and Natural Gas Corporation'],
    'POWERGRID': ['POWERGRID', 'Power Grid Corporation'],
    'RELIANCE': ['RELIANCE', 'Reliance Industries'],
    'SBIN': ['SBIN', 'State Bank of India'],
    'SHREECEM': ['SHREECEM', 'Shree Cement'],
    'SUNPHARMA': ['SUNPHARMA', 'Sun Pharma'],
    'TATAMOTORS': ['TATAMOTORS', 'Tata Motors'],
    'TATASTEEL': ['TATASTEEL', 'Tata Steel'],
    'TCS': ['TCS', 'Tata Consultancy Services'],
    'TECHM': ['TECHM', 'Tech Mahindra'],
    'TITAN': ['TITAN', 'Titan Company'],
    'ULTRACEMCO': ['ULTRACEMCO', 'UltraTech Cement'],
    'UPL': ['UPL', 'UPL Limited'],
    'VEDL': ['VEDL', 'Vedanta'],
    'WIPRO': ['WIPRO', 'Wipro Limited'],
    'ZEEL': ['ZEEL', 'Zee Entertainment']
}
companies = {}

for stock, names in stock_companies.items():
  companies[stock]=[name.lower() for name in names]



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

import json
with open('/content/drive/MyDrive/articles/content1.json', 'r') as f:
  articles = json.load(f)

In [ ]:
from tqdm import tqdm
import regex as re

company_articles = []

i=0
for article in tqdm(articles, desc="Processing articles"):
    title = article['title'].lower()
    content = article['content'].lower()
    for company, names in companies.items():
        for name in names:
            pattern = r'\b{}\b'.format(re.escape(name.lower()))
            if re.search(pattern, title) or re.search(pattern, content):
                company_articles.append({
                    'date': article['date'],
                    'company': company,
                    'content': article['content']
                })
                break

Processing articles: 100%|██████████| 27529/27529 [01:01<00:00, 444.06it/s]


In [ ]:
print(len(company_articles), len(articles))

max_len = 0
for article in company_articles:
  if len(article['content']) > max_len:
    max_len = len(article['content'])
print(max_len)

10354 27529
19903


In [ ]:
# Approach 1
import os
import torch
from transformers import pipeline, logging
from tqdm import tqdm
import json

logging.set_verbosity_error()

# Initialize the summarizer once
summarizer = pipeline(
    'summarization',
    'pszemraj/led-large-book-summary',
    device=0 if torch.cuda.is_available() else -1,
    model_kwargs={'max_length': 700}
)

processed = []
if os.path.exists('/content/drive/MyDrive/articles/summaries1.json'):
    with open('/content/drive/MyDrive/articles/summaries1.json', 'r') as f:
      processed = json.load(f)

# Define batch summarization function
def summarize_articles(articles, batch_size=6):
    summaries = []

    for i in tqdm(range(len(processed), len(articles), batch_size), desc="Summarizing articles"):
        batch = [article['content'] for article in articles[i:i+batch_size]]
        results = summarizer(batch)
        for article, summary in zip(articles[i:i+batch_size], results):
            article['content'] = summary['summary_text']
            processed.append(article)
        with open('/content/drive/MyDrive/articles/summaries1.json', 'w') as f:
            json.dump(processed, f)
    return summaries

# Summarize articles in batches
summarize_articles(company_articles)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.84G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

Summarizing articles:   8%|▊         | 133/1726 [1:20:58<11:37:31, 26.27s/it]

In [ ]:
# Approach 2
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, logging
from tqdm import tqdm
import json

logging.set_verbosity_error()

# Set up device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load tokenizer and model
model_name = 'pszemraj/led-large-book-summary'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

# Function to summarize a batch of articles
def summarize_batch(contents):
    inputs = tokenizer(
        contents,
        return_tensors='pt',
        max_length=16384,
        truncation=True,
        padding='longest'
    )
    input_ids = inputs.input_ids.to(device)
    attention_mask = inputs.attention_mask.to(device)
    with torch.no_grad():
        summaries = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_length=600,
            num_beams=4,
            length_penalty=2.0,
            no_repeat_ngram_size=3
        )
    decoded_summaries = [tokenizer.decode(g, skip_special_tokens=True) for g in summaries]
    return decoded_summaries

# Prepare contents and set batch size
contents = [article['content'] for article in company_articles]
batch_size = 8  # Based on GPU capacity

# Summarize articles in batches
summarized_articles = []
for i in tqdm(range(0, len(contents), batch_size), desc="Summarizing articles"):
    batch_contents = contents[i:i+batch_size]
    batch_summaries = summarize_batch(batch_contents)
    for article, summary in zip(company_articles[i:i+batch_size], batch_summaries):
        article['summary'] = summary
        summarized_articles.append(article)

# Save the summarized articles
with open('/content/drive/MyDrive/articles/summaries1.json', 'w') as f:
    json.dump(summarized_articles, f)

In [ ]:
stock_companies = {
    'ADANIPORTS': ['ADANIPORTS', 'Adani Ports'],
    'ASIANPAINT': ['ASIANPAINT', 'Asian Paints'],
    'AXISBANK': ['AXISBANK', 'Axis Bank'],
    'BAJAJ-AUTO': ['BAJAJ-AUTO', 'Bajaj Auto'],
    'BAJAJFINSV': ['BAJAJFINSV', 'Bajaj Finserv'],
    'BAJFINANCE': ['BAJFINANCE', 'Bajaj Finance'],
    'BHARTIARTL': ['BHARTIARTL', 'Bharti Airtel', 'Airtel'],
    'BRITANNIA': ['BRITANNIA', 'Britannia Industries'],
    'BPCL': ['BPCL', 'Bharat Petroleum'],
    'COALINDIA': ['COALINDIA', 'Coal India'],
    'EICHERMOT': ['EICHERMOT', 'Eicher Motors'],
    'DRREDDY': ['DRREDDY', "Dr. Reddy's Laboratories"],
    'GAIL': ['GAIL', 'Gas Authority of India'],
    'GRASIM': ['GRASIM', 'Grasim Industries'],
    'CIPLA': ['CIPLA', 'Cipla Limited'],
    'HCLTECH': ['HCLTECH', 'HCL Technologies'],
    'HDFCBANK': ['HDFCBANK', 'HDFC Bank'],
    'HEROMOTOCO': ['HEROMOTOCO', 'Hero MotoCorp'],
    'HINDUNILVR': ['HINDUNILVR', 'Hindustan Unilever', 'HUL'],
    'HINDALCO': ['HINDALCO', 'Hindalco Industries'],
    'ICICIBANK': ['ICICIBANK', 'ICICI Bank'],
    'INDUSINDBK': ['INDUSINDBK', 'IndusInd Bank'],
    'INFY': ['INFY', 'Infosys'],
    'IOC': ['IOC', 'Indian Oil'],
    'ITC': ['ITC', 'ITC Limited'],
    'JSWSTEEL': ['JSWSTEEL', 'JSW Steel'],
    'KOTAKBANK': ['KOTAKBANK', 'Kotak Mahindra Bank'],
    'LT': ['LT', 'Larsen & Toubro'],
    'MARUTI': ['MARUTI', 'Maruti Suzuki'],
    'MM': ['MM', 'Mahindra & Mahindra'],
    'NESTLEIND': ['NESTLEIND', 'Nestle India'],
    'NTPC': ['NTPC', 'NTPC Limited'],
    'ONGC': ['ONGC', 'Oil and Natural Gas Corporation'],
    'POWERGRID': ['POWERGRID', 'Power Grid Corporation'],
    'RELIANCE': ['RELIANCE', 'Reliance Industries'],
    'SBIN': ['SBIN', 'State Bank of India'],
    'SHREECEM': ['SHREECEM', 'Shree Cement'],
    'SUNPHARMA': ['SUNPHARMA', 'Sun Pharma'],
    'TATAMOTORS': ['TATAMOTORS', 'Tata Motors'],
    'TATASTEEL': ['TATASTEEL', 'Tata Steel'],
    'TCS': ['TCS', 'Tata Consultancy Services'],
    'TECHM': ['TECHM', 'Tech Mahindra'],
    'TITAN': ['TITAN', 'Titan Company'],
    'ULTRACEMCO': ['ULTRACEMCO', 'UltraTech Cement'],
    'UPL': ['UPL', 'UPL Limited'],
    'VEDL': ['VEDL', 'Vedanta'],
    'WIPRO': ['WIPRO', 'Wipro Limited'],
    'ZEEL': ['ZEEL', 'Zee Entertainment']

}

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
# hehehe
url = "https://finance.yahoo.com/quote/GC%3DF/history/?period1=940702043&period2=1733314816"
comp = "GOLD"
r = requests.get(url, headers=headers)
soup = BeautifulSoup(r.text, 'html.parser')
data_div = soup.find("div", {"data-testid": "history-table"})

table = soup.find("table", {"class": "table yf-j5d1ld"})

if table:
    # Extract table data
    data = []
    headers = [header.text for header in table.find_all("th")]
    for row in table.find_all("tr")[1:]: # Skip header row
        cols = row.find_all("td")
        if cols: # Check if the row contains data
          row_data = [col.text.strip() for col in cols]
          data.append(row_data)

    # Create a Pandas DataFrame
    df = pd.DataFrame(data, columns=headers)
    df.columns = ['Date', 'Open', 'High', 'Low','Close','Adj Close','Volume']
    df['Date'] = pd.to_datetime(df['Date'], format='%b %d, %Y')
    df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')
    df = df.iloc[::-1]
    print(df)
    # Save data to CSV file
    df.to_csv(f'/content/drive/MyDrive/articles/{comp}.csv', index=False)
    print(f"Data saved to {comp}.csv")

else:
    print("Table not found on the page.")
    print(data_div)

            Date    Open    High     Low   Close Adj Close      Volume
5591  2002-07-01   38.85   39.14   38.48   38.88     32.09   4,031,528
5590  2002-07-02   38.73   40.12   38.54   39.21     32.37  14,152,321
5589  2002-07-03   39.03   39.76   38.86   39.57     32.66   8,456,844
5588  2002-07-04   39.87   40.83   39.31   39.54     32.64  12,745,157
5587  2002-07-05   40.01   40.01   39.02   39.35     32.48   6,445,895
...          ...     ...     ...     ...     ...       ...         ...
4     2024-11-28  122.70  124.79  122.07  123.11    123.11   6,700,108
3     2024-11-29  129.00  132.69  127.50  129.16    129.16  46,663,653
2     2024-12-02  130.00  131.50  129.51  130.51    130.51  12,878,159
1     2024-12-03  131.10  139.00  130.64  138.17    138.17  35,419,356
0     2024-12-04  138.95  140.30  136.95  137.70    137.70   9,282,623

[5592 rows x 7 columns]
Data saved to ZEEL.csv
